### Import

In [5]:
import cv2
import IPython
import numpy as np
from io import BytesIO
import matplotlib.pyplot as plt
from typing import List, Dict, Tuple, NoReturn

### Params

- KNOWN_DISTANCE - Known distance from the reference image *Inches
- PERSON_WIDTH - Person avg width in real life *Inches
- CAR_WIDTH - Car width in real life *Inches
- CONFIDENCE_THRESHOLD - Minimal threshold to detect object 
- NMS_THRESHOLD - Minimal NMS to suppress the less likely bounding boxes by score & IOU
- NET_IMG_SIZE - Preferable image size for net
- BLACK_RGB - RGB Value for black
- FIG_SIZE - Streaming figsize view

In [6]:
KNOWN_DISTANCE = 100.0
PERSON_WIDTH = 16.0
CAR_WIDTH = 100.0
CONFIDENCE_THRESHOLD = 0.4
NMS_THRESHOLD = 0.3
NET_IMG_SIZE = (512, 512)
INCH_TO_METER = 0.0254
BLACK_RGB = (0, 0, 0)
FIG_SIZE = (10,10)

### NetParams

- Currently used pretrined YoloV4 weights with full classes

In [7]:
yoloNet = cv2.dnn.readNet('./resource/yolov4-tiny.weights', './resource/yolov4-tiny.cfg')
yoloNet.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
yoloNet.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA_FP16)

model = cv2.dnn_DetectionModel(yoloNet)
model.setInputParams(size=NET_IMG_SIZE, scale=1/255, swapRB=True)

with open("./resource/classes.txt", "r") as f:
    names = [name.strip() for name in f.readlines()]

### Distance measurement

- focal_distance - ( reference_width * known distance from the object ) / real object width
- distance_finder - ( focal_distance * real object width ) / object width in frame 

### Object detection

- predict - predict objects & draw rectangle

### Preview

- display - display video streaming live

In [8]:
def predict(image: np.ndarray) -> List[list]:
    data_list =[]
    classes, scores, boxes = model.detect(image, CONFIDENCE_THRESHOLD, NMS_THRESHOLD)

    for (classid, score, box) in zip(classes, scores, boxes):    
        
        cv2.rectangle(image, box, BLACK_RGB, 2)
        cv2.putText(image, str(names[classid]), (box[0], box[1]-14), cv2.FONT_HERSHEY_COMPLEX, 1, BLACK_RGB, 2)
    
        if classid == 0: # person
            data_list.append([names[classid], box[2], (box[0], box[1]-2)])
            
        if classid == 2: # car  
            data_list.append([names[classid], box[2], (box[0], box[1]-2)])
        
    return data_list


def focal_distance(measured_distance: float, real_width: float, width_in_rf: float) -> float:
    return (measured_distance * width_in_rf) / real_width


def distance_finder(focal_length : float, real_object_width: float, width_in_frmae: float) -> float:
    return (focal_length * real_object_width) / width_in_frmae


def display(frame: np.ndarray) -> NoReturn:
    plt.figure(figsize=FIG_SIZE)
    plt.imshow(frame)
    f = BytesIO()
    plt.savefig(f, format='jpeg'), plt.close()
    IPython.display.display(IPython.display.Image(data=f.getvalue()))

### Reference params

- Used to determine detected object size to the real world object
- Currently calculated for person & cars/trucks

In [10]:
ref_person = cv2.imread('./images/sample_person.png')
ref_car = cv2.imread('./images/sample_car.png')

person_data = predict(ref_person)
person_width_in_rf = person_data[0][1]
focal_person = focal_distance(KNOWN_DISTANCE, PERSON_WIDTH, person_width_in_rf)

car_data = predict(ref_car)
car_width_in_rf = car_data[0][1]
focal_car = focal_distance(KNOWN_DISTANCE, CAR_WIDTH, car_width_in_rf)

### Stream

In [ ]:
cap = cv2.VideoCapture('./images/sample1.mp4')

while True:
    ret, frame = cap.read()
    data, distance = predict(frame), 0
    
    for d in data:
        if d[0] =='person':
            distance = distance_finder(focal_person, PERSON_WIDTH, d[1]) * INCH_TO_METER

        if d[0] == 'car':
            distance = distance_finder(focal_car, CAR_WIDTH, d[1]) * INCH_TO_METER

        x, y = d[2]
        cv2.putText(frame, str(round(distance,1)), (x+5,y+13), cv2.FONT_HERSHEY_COMPLEX, 0.8, BLACK_RGB , 2)

    display(frame)
    IPython.display.clear_output(wait=True)